In [59]:

from __future__ import annotations

import sys, random
from pathlib import Path
from typing import Optional, Tuple, Union
from scipy.ndimage import median_filter     # std-lib from SciPy ≥ 1.6
import numpy as np
import pandas as pd



# ---------------------------------------------------------------------------
# Optional KeplerGL inline display
# ---------------------------------------------------------------------------
try:
    from keplergl import KeplerGl  # type: ignore
    from IPython.display import display  # type: ignore

    _HAS_KEPLER = True
except ImportError:
    _HAS_KEPLER = False

# ---------------------------------------------------------------------------
NM_IN_DEG = 1.0 / 60.0  # ≈ 1 NM in degrees latitude at the equator


def _delta_lat_lon(lat: float, bearing_deg: float, dist_nm: float) -> Tuple[float, float]:
    """Small‑displacement Δlat/Δlon (deg) for given bearing and distance (NM)."""
    d = dist_nm * NM_IN_DEG
    rad = np.deg2rad(bearing_deg)
    return d * np.cos(rad), d * np.sin(rad) / np.cos(np.deg2rad(lat))




In [60]:
# USER CONFIG – edit and run the cell
# ---------------------------------------------------------------------------
SCRIPT_PATH = Path.cwd().parent

EXISTING_ABNO = SCRIPT_PATH / "data" / "new_abnormality_collection.csv"
INPUT_PATH = SCRIPT_PATH / "data" / "Final_data.csv"


In [61]:
chunk_size = 500000

try:
    base_chunks = pd.read_csv(INPUT_PATH, parse_dates=["# Timestamp"], chunksize=chunk_size)
    base = pd.concat(base_chunks)
except pd.errors.EmptyDataError:
    print("INPUT_CSV is empty!")
    base = pd.DataFrame()

print(base.shape)
print(base.columns)




(4335995, 8)
Index(['# Timestamp', 'MMSI', 'Latitude', 'Longitude', 'ROT', 'SOG', 'COG',
       'Heading'],
      dtype='object')


In [62]:
TARGET_MMSI = 636020765
START_TIME = "2025-02-07 02:23:42"
''''



636093236 zigzag
626281000 zigzag
538009200 pickup
257114000 deviation
305286000 deviation 
636018261 zigzag
257186000 pickup
636020106 deviation
219945000 zigzag
538002778 deviation
636092635 pickup
636022249 zigzag
266465000 deviation
304717000 pickup
257064430 zigzag
230601000 deviation
305773000 pickup
538010240 zigzag
255915583 deviation
265859000 pickup (-700 ROT)
265177000 zigzag
314826000 deviation
258003290 pickup
257207000 zigzag
538010456 deviation

265410000 pickup
563019500 zigzag
275510000 deviation
636022149 pickup
636020765 zigzag


'''

CONFIG = {
    "INPUT_CSV": SCRIPT_PATH / "output" / "single_ship.csv", #Path til valgt
    "OUTPUT_CSV": SCRIPT_PATH / "output" / "anomaly_edit.csv",
    "SEED": 42, # Bare til random generation
    "ANOMALY": "zigzag",   # deviation | speed_spike | stop | pickup | loop | zigzag | anchor_drift | gap | random (SKRIV HVILKE ANOMALY DU VIL HA)
    "PCT_SEGMENT": 0.08,    # 0.08 er default. Hvor meget af tracket der skal bruges til en abnormalitet. 
    "START_TS": START_TIME,  # Random virker ikke så vælg et punkt
    "INTENSITY": 2,       # 1.0 = default, >1 = stronger, <1 = milder
    "TARGET_MMSI": TARGET_MMSI,  
}





In [63]:
try:
    df_existing = pd.read_csv(EXISTING_ABNO)
except pd.errors.EmptyDataError:
    print("EXISTING_ABNORMALITIES is empty!. Creating new empty DataFrame.")
    df_existing = pd.DataFrame(columns=[ "# Timestamp","MMSI","Latitude","Longitude","ROT","SOG","COG","Heading"])

In [64]:
# ---------------------------------------------------------------------------
class AISAnomalyInjector:
    """Injects a single labelled anomaly segment into a ship track."""

    REQ = {"# Timestamp","MMSI","Latitude","Longitude","ROT","SOG","COG","Heading"}

    def __init__(self, df: pd.DataFrame, seed: Optional[int] = None):
        missing = self.REQ - set(df.columns)
        if missing:
            raise ValueError(f"AIS dataframe missing required columns: {missing}")
        self.df = df.sort_values("# Timestamp").reset_index(drop=True)
        self.rng = random.Random(seed)
        self.intensity: float = 1.0  # will be overwritten in `.inject()`

    # ------------------------------------------------------------------
    def inject(
        self,
        kind: str = "random",
        pct: float = 0.1,
        start_ts: Optional[Union[str, pd.Timestamp]] = None,
        intensity: float = 1.0,
    ) -> pd.DataFrame:
        """Inject *one* anomaly of the given kind into the track.

        Parameters
        ----------
        kind : str
            Anomaly type; "random" picks one at random.
        pct : float
            Fraction of the track to affect (window length).
        start_ts : str | pd.Timestamp | None
            Start timestamp for the anomaly window, or *None* → random start.
        intensity : float, default 1.0
            1.0 reproduces legacy behaviour; >1 exaggerates, <1 attenuates.
        """
        if kind == "random":
            kind = self.rng.choice([
                "deviation", "speed_spike", "stop", "pickup", "loop", "zigzag", "anchor_drift", "gap",
            ])
        self.intensity = max(0, float(intensity))  # clip at 0

        out = self.df.copy()
        out["anomaly_type"] = np.nan

        s, e = self._pick_window(pct, start_ts)
        a = out.loc[s, ["Latitude", "Longitude"]].to_numpy(float)
        b = out.loc[e, ["Latitude", "Longitude"]].to_numpy(float)

        getattr(self, f"_do_{kind}")(out, s, e, a, b)

        # snap endpoints back exactly
        out.loc[s, ["Latitude", "Longitude"]] = a
        out.loc[e, ["Latitude", "Longitude"]] = b

        self._recompute_kinematics(out, s, e) # Gør så koden ikke dividere med 0, fikser nogle store spikes og smoother den abnormale data ud
        self._blend_sog(out, s, e)  # Denne får SOG til at matche den abnormale data med den originale data før og efter abnormaliteten.
        self._insert_heading(out, s, e) # Fikser heading, så den også er realistisk
        self._round_precision(out, s, e) # Runder numrene op så det matcher rigtig data.

        return out

    # ------------------------------------------------------------------
    # Anomaly generators (modified to honour self.intensity)
    # ------------------------------------------------------------------

    def _do_deviation(self, df, s, e, a, b):
        n = e - s + 1
        # 1) baseline
        lat_base = np.linspace(a[0], b[0], n)
        lon_base = np.linspace(a[1], b[1], n)
        # 2) one perp offset
        avg_lat = (a[0] + b[0]) / 2
        bearing_ab = np.rad2deg(np.arctan2(
            (b[1] - a[1]) * np.cos(np.deg2rad(avg_lat)),
            (b[0] - a[0])
        ))

        perp_brg = (bearing_ab + 90) % 360
        amp_nm = self.rng.uniform(1.0, 5.0) * self.intensity
        dlat, dlon = _delta_lat_lon(a[0], perp_brg, amp_nm)
        # 3) smooth ramp in/out
        ramp = np.sin(np.linspace(0, np.pi, n))
        # 4) write
        df.loc[s:e, "Latitude"]  = lat_base + dlat * ramp
        df.loc[s:e, "Longitude"] = lon_base + dlon * ramp
        df.loc[s:e, "anomaly_type"] = "deviation"

    def _do_speed_spike(self, df, s, e, *_):
        n = e - s + 1
        base = self.rng.uniform(2, 4) * np.sin(np.linspace(0, np.pi, n)) ** 2 + 0.5
        mult = 1 + (base - 1) * self.intensity  # scale away from 1
        df.loc[s:e, "SOG"] *= mult
        df.loc[s:e, "ROT"] += self.rng.uniform(-5, 5) * self.intensity * np.random.randn(n)
        df.loc[s:e, "anomaly_type"] = "speed_spike"

    def _do_stop(self, df, s, e, a, b):
        """
        Simulate a temporary stop: vessel smoothly slows to (almost) zero speed,
        then accelerates back to normal.  Position follows the straight line
        a→b so the track stays continuous; only the kinematics show the stop.

        INTENSITY > 1 widens the low‑speed plateau, < 1 narrows it.
        """
        n = e - s + 1

        # --- keep positions on the baseline straight segment ---
        df.loc[s:e, "Latitude"]  = np.linspace(a[0], b[0], n)
        df.loc[s:e, "Longitude"] = np.linspace(a[1], b[1], n)

        # --- speed profile: bell‑shaped slowdown to ~0 ---
        ramp = np.sin(np.linspace(0, np.pi, n))            # 0→1→0
        min_fac = 0.05                                     # 5 % of original SOG at full stop
        # Scale ramp duration with intensity (>1 = wider stop, <1 = narrower)
        width = np.clip(self.intensity, 0.1, 10)           # avoid div‑by‑0 / absurd spikes
        fac = 1 - (1 - min_fac) * np.clip(ramp * width, 0, 1)
        df.loc[s:e, "SOG"] *= fac

        # --- little to no turning while stopped ---
        df.loc[s:e, "ROT"] *= 0.1

        df.loc[s:e, "anomaly_type"] = "stop"

    def _do_pickup(self, df, s, e, a, b):
        n = e - s + 1
        lat_base = np.linspace(a[0], b[0], n)
        lon_base = np.linspace(a[1], b[1], n)

        amp_nm = self.rng.uniform(1.0, 5.0) * self.intensity
        side   = self.rng.choice([-1, 1])
        avg_lat = (a[0] + b[0]) / 2
        bearing_ab = np.rad2deg(np.arctan2(
            (b[1] - a[1]) * np.cos(np.deg2rad(avg_lat)),
            (b[0] - a[0])
        ))
        perp_brg = (bearing_ab + 90 * side) % 360
        dlat, dlon = _delta_lat_lon(a[0], perp_brg, amp_nm)

        # build a 3‑stage ramp: 0→1, hold, 1→0
        ramp = np.zeros(n)
        t1 = n // 3
        t2 = 2 * n // 3
        ramp[:t1]   = np.linspace(0, 1, t1)
        ramp[t1:t2] = 1
        ramp[t2:]   = np.linspace(1, 0, n - t2)
        # soften
        ramp = np.sin(ramp * np.pi / 2)

        df.loc[s:e, "Latitude"]  = lat_base + dlat * ramp
        df.loc[s:e, "Longitude"] = lon_base + dlon * ramp
        df.loc[s:e, "anomaly_type"] = "pickup"

    def _do_loop(self, df, s, e, a, b):
        n = e - s + 1
        # baseline
        lat_base = np.linspace(a[0], b[0], n)
        lon_base = np.linspace(a[1], b[1], n)
        # unit‑vector along a→b (in deg‑space)
        avg_lat = (a[0] + b[0]) / 2
        dx = b[0] - a[0]
        dy = (b[1] - a[1]) * np.cos(np.deg2rad(avg_lat))
        dist = np.hypot(dx, dy) or 1.0
        ux, uy = dx / dist, dy / dist
        # perp unit
        px, py = -uy, ux
        # loop radius
        r_nm = self.rng.uniform(0.2, 0.5) * self.intensity
        off = r_nm * NM_IN_DEG
        # angle for 1→1.5 loops
        θ = np.linspace(0, 2 * np.pi * self.rng.uniform(1, 1.5), n)
        # compute offsets
        lat_off = px * off * np.sin(θ)
        lon_off = py * off * np.sin(θ) / np.cos(np.deg2rad(lat_base))
        df.loc[s:e, "Latitude"]  = lat_base + lat_off
        df.loc[s:e, "Longitude"] = lon_base + lon_off
        df.loc[s:e, "anomaly_type"] = "loop"

    def _do_zigzag(self, df, s, e, a, b):
        n = e - s + 1
        # baseline
        lat_base = np.linspace(a[0], b[0], n)
        lon_base = np.linspace(a[1], b[1], n)
        # pick # of half‑waves (even so endpoints zero)
        waves = self.rng.randint(2, 8)
        θ = np.linspace(0, waves * np.pi, n)
        # lateral distance
        offset_nm = self.rng.uniform(0.1, 0.3) * self.intensity
        avg_lat = (a[0] + b[0]) / 2
        # build perp vector (same as deviation)
        bearing_ab = np.rad2deg(np.arctan2(
            (b[1] - a[1]) * np.cos(np.deg2rad(avg_lat)),
            (b[0] - a[0])
        ))
        perp_brg = (bearing_ab + 90) % 360
        dlat, dlon = _delta_lat_lon(a[0], perp_brg, offset_nm)
        # sinusoidal zigzag
        sinz = np.sin(θ)
        df.loc[s:e, "Latitude"]  = lat_base + dlat * sinz
        df.loc[s:e, "Longitude"] = lon_base + dlon * sinz
        df.loc[s:e, "anomaly_type"] = "zigzag"

    def _do_anchor_drift(self, df, s, e, a, b):
        n = e - s + 1
        # baseline
        lat_base = np.linspace(a[0], b[0], n)
        lon_base = np.linspace(a[1], b[1], n)
        # small steady drift off one side
        amp_nm = self.rng.uniform(0.05, 0.2) * self.intensity
        avg_lat = (a[0] + b[0]) / 2
        bearing_ab = np.rad2deg(np.arctan2(
            (b[1] - a[1]) * np.cos(np.deg2rad(avg_lat)),
            (b[0] - a[0])
        ))
        perp_brg = (bearing_ab + 90) % 360
        dlat, dlon = _delta_lat_lon(a[0], perp_brg, amp_nm)
        # smooth ramp in/out just like deviation
        ramp = np.sin(np.linspace(0, np.pi, n))
        df.loc[s:e, "Latitude"]  = lat_base + dlat * ramp
        df.loc[s:e, "Longitude"] = lon_base + dlon * ramp
        # keep it drifting slowly
        df.loc[s:e, "SOG"] = self.rng.uniform(0.1, 0.5) * self.intensity
        df.loc[s:e, "anomaly_type"] = "anchor_drift"

    def _do_gap(self, df, s, e, a, b):
        n = e - s + 1
        df.loc[s:e, "Latitude"]  = np.linspace(a[0], b[0], n)
        df.loc[s:e, "Longitude"] = np.linspace(a[1], b[1], n)
        df.loc[s:e, ["ROT", "COG", "SOG"]] = np.nan
        df.loc[s:e, "anomaly_type"] = "gap"

    # ------------------------------------------------------------------
    def _recompute_kinematics(self, df, s, e):
        """
        Overwrite COG, SOG, ROT for rows s…e so they reflect the
        edited lat / lon.  Uses simple finite-differences in NM & sec.
        """
        # 1. helpers ----------------------------------------------------
        def _bearing(lat1, lon1, lat2, lon2):
            x = np.deg2rad(lon2 - lon1) * np.cos(np.deg2rad((lat1 + lat2) / 2))
            y = np.deg2rad(lat2 - lat1)
            brg = (np.rad2deg(np.arctan2(x, y)) + 360) % 360
            return brg

        EARTH_NM = 60 * 180 / np.pi          # 1 rad ≈ 60 NM
        def _dist_nm(lat1, lon1, lat2, lon2):
            dφ = np.deg2rad(lat2 - lat1)
            dλ = np.deg2rad(lon2 - lon1)
            a = np.sin(dφ/2)**2 + np.cos(np.deg2rad(lat1))*np.cos(np.deg2rad(lat2))*np.sin(dλ/2)**2
            return 2 * EARTH_NM * np.arcsin(np.sqrt(a))

        # 2. slice we need ---------------------------------------------
        lat  = df["Latitude"].to_numpy()
        lon  = df["Longitude"].to_numpy()
        ts   = df["# Timestamp"].to_numpy(dtype='datetime64[ns]').astype('datetime64[s]').astype(int)

        # idx + 1 because we need fwd diff; last row can reuse previous value
        sl = slice(s, e)          # s … e-1
        nxt = slice(s+1, e+1)     # s+1 … e

        # --- choose exclusive slice s:e with .iloc so lengths match -------------
        # COG --------------------------------------------------------------------
        df.iloc[s:e, df.columns.get_loc("COG")] = _bearing(
            lat[s:e], lon[s:e],
            lat[s+1:e+1], lon[s+1:e+1]
        )

        # SOG --------------------------------------------------------------------
        dist_nm  = _dist_nm(lat[s:e], lon[s:e], lat[s+1:e+1], lon[s+1:e+1])
        dt_hours = (ts[s+1:e+1] - ts[s:e]) / 3600          # sec → h
        df.iloc[s:e, df.columns.get_loc("SOG")] = dist_nm / dt_hours

        # ROT --------------------------------------------------------------------
        cog_rad = np.deg2rad(df["COG"].to_numpy())
        dcog    = np.unwrap(cog_rad)[s+1:e+1] - np.unwrap(cog_rad)[s:e]
        df.iloc[s:e, df.columns.get_loc("ROT")] = (
            np.rad2deg(dcog) / ((ts[s+1:e+1] - ts[s:e]) / 60)   # deg / min
        )

    def _blend_sog(self, df, s, e, window=5):
        """
        Linearly warps the *new* SOG values in rows s … e-1 so that they
        join the original SOG that still exists in rows s-1 and e.
        Also applies a short rolling-median to remove residual spikes.
        """
        # indices for convenience
        edited   = slice(s, e)          # rows with synthetic track
        n        = e - s                # length of edited segment
        if n <= 0:
            return                      # nothing to do

        # endpoint targets (the 'real' AIS values outside the splice)
        sog_left  = df.at[s-1, "SOG"] if s > 0 else np.nan
        sog_right = df.at[e,   "SOG"] if e < len(df) else np.nan

        # synthetic SOG that came out of _recompute_kinematics
        sog_new = df.loc[edited, "SOG"].to_numpy(dtype=float)

        # If either neighbour is missing (NaN) just median-filter and quit
        if np.isnan(sog_left) or np.isnan(sog_right):
            df.loc[edited, "SOG"] = median_filter(sog_new, size=window,
                                                mode="nearest")
            return

        # -------- affine warp so that first and last points hit the targets ----
        # y = a * x + b  ;  solve so that y0 = sog_left,  y_{n-1} = sog_right
        a = (sog_right - sog_left) / (sog_new[-1] - sog_new[0] \
                                    + 1e-9)          # avoid /0
        b = sog_left - a * sog_new[0]
        sog_blended = a * sog_new + b

        # optional light smoothing
        sog_smoothed = median_filter(sog_blended, size=window, mode="nearest")

        # clip back into AIS range (0 … 102.2 kt) and write back
        df.loc[edited, "SOG"] = np.clip(sog_smoothed, 0, 102.2)


    def _insert_heading(self, df, s, e, window=7):
        """
        Fill / overwrite the HDG column in rows s … e-1 so that it follows
        the new path, then lightly smooth it.
        """
        edited = slice(s, e)
        if "Heading" not in df.columns:
            df.insert(df.columns.get_loc("COG") + 1, "Heading", np.nan)

        # reuse the already unwrapped COG series (created in _recompute_kinematics)
        hdg = df.loc[edited, "COG"].to_numpy(dtype=float)

        # centre-aligned Savitzky–Golay filter (window must be odd)
        from scipy.signal import savgol_filter
        if window % 2 == 0:
            window += 1                 # make it odd
        hdg_smooth = savgol_filter(hdg, window_length=window, polyorder=2,
                                mode="interp")

        # clip into valid range 0 … 359.9 and write back
        df.loc[edited, "Heading"] = np.mod(hdg_smooth, 360)


    def _round_precision(self, df, s, e):
        """
        Round the freshly edited rows s … e-1 to AIS-style precision.

            • ROT, SOG, COG  → 1 decimal  (e.g. 23.1)
            • Latitude, Longitude → 6 decimals (e.g. 11.828833)
            • Heading → integer 0-359
        """
        edited = slice(s, e)

        # 1-dp for the rates and courses
        cols1dp = ["ROT", "SOG", "COG"]
        df.loc[edited, cols1dp] = df.loc[edited, cols1dp].round(1)

        # 6-dp for position
        df.loc[edited, ["Latitude", "Longitude"]] = (
            df.loc[edited, ["Latitude", "Longitude"]].round(6)
        )

        # integer heading (use rint to avoid -0.0)
        df.loc[edited, "Heading"] = (
            np.rint(df.loc[edited, "Heading"]).astype("Int64")   # keeps NaN as <NA>
        )




    # ------------------------------------------------------------------
    def _pick_window(self, pct: float, start_ts: Optional[Union[str, pd.Timestamp]]):
        """Return start & end indices for the anomaly window."""
        L = len(self.df)
        span = max(3, int(L * pct))
        if start_ts is not None:
            ts = pd.Timestamp(start_ts)
            idx = self.df.index[self.df["# Timestamp"] >= ts]
            if idx.empty:
                raise ValueError("START_TS is after the data ends")
            s = idx[0]
            e = min(s + span, L - 1)
            if e - s < 2:
                raise ValueError("START_TS too close to track end for PCT_SEGMENT")
            return s, e

        # --- random‑start mode ---
        s = self.rng.randint(0, L - span - 1)
        return s, s + span


# ---------------------------------------------------------------------------
# Kepler map helper
# ---------------------------------------------------------------------------
def show_maps(before: pd.DataFrame, after: pd.DataFrame):
    """Inline before/after layers if keplergl is available."""
    if not _HAS_KEPLER:
        print("[!] keplergl not installed – skipping map display")
        return
    display(KeplerGl(height=400, data={"before": before}))
    display(KeplerGl(height=400, data={"after": after}))


# ---------------------------------------------------------------------------
# Runner helper
# ---------------------------------------------------------------------------
def run(cfg: dict, df):
    dst = Path(cfg["OUTPUT_CSV"])
    src = INPUT_PATH
    base = df
    
    base = base[base["MMSI"]==TARGET_MMSI]


    injector = AISAnomalyInjector(base, seed=cfg["SEED"])
    augmented = injector.inject(cfg["ANOMALY"],
                                cfg["PCT_SEGMENT"],
                                cfg["START_TS"],
                                cfg["INTENSITY"])

    show_maps(base, augmented)
    dst.parent.mkdir(parents=True, exist_ok=True)
    augmented = augmented.drop("anomaly_type", axis=1)
    # MY SHIT (AUGMENT INDICATOR AND RANDOM MMSI)
    augmented["Label"] = True

    #new_mmsi = random.randint(100000000, 999999999)
    #print(f"New mmsi: {new_mmsi}")
    #augmented["MMSI"] = new_mmsi
    
    # CHECK FOR EXCISTING MMSI IN AUG CSV FILE
    if TARGET_MMSI not in df_existing['MMSI'].values:
        df_updated = pd.concat([df_existing, augmented], ignore_index=True)
        df_updated.to_csv(EXISTING_ABNO, index=False)
        print(f"[✓] CSV written → {dst.relative_to(SCRIPT_PATH)}")

    else:
        print("Boat with that MMSI already exists. Skipped adding.")


    return augmented


# ---------------------------------------------------------------------------
# Entry‑point  –  Jupyter uses CONFIG, shell can use CLI
# ---------------------------------------------------------------------------
if __name__ == "__main__":
    # If at least two positional args AND first arg isn’t an option, treat as CLI
    if len(sys.argv) >= 3 and not sys.argv[1].startswith("-"):
        import argparse

        p = argparse.ArgumentParser(description="Inject AIS anomaly and show maps")
        p.add_argument("input_csv")
        p.add_argument("output_csv")
        p.add_argument("--start")
        p.add_argument("--pct",  type=float, default=CONFIG["PCT_SEGMENT"])
        p.add_argument("--kind", default="random")
        p.add_argument("--seed", type=int,    default=CONFIG["SEED"])
        # (INTENSITY flag deliberately left out to keep CLI unchanged unless requested)
        args = p.parse_args()

        cfg = CONFIG.copy()
        cfg.update(INPUT_CSV=Path(args.input_csv),
                   OUTPUT_CSV=Path(args.output_csv),
                   START_TS=args.start,
                   PCT_SEGMENT=args.pct,
                   ANOMALY=args.kind,
                   SEED=args.seed)
        run(cfg, df = base)
    else:
        print("Running with CONFIG dict – edit at top as needed.\n")
        run(CONFIG, df = base)

# END OF CODE


Running with CONFIG dict – edit at top as needed.

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


C:\Users\rasmu\AppData\Local\Temp\ipykernel_14980\2113581059.py:199: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'zigzag' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[s:e, "anomaly_type"] = "zigzag"


KeplerGl(data={'before':                 # Timestamp       MMSI   Latitude  Longitude  ROT   SOG  \
3566868 20…

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


C:\Users\rasmu\AppData\Roaming\Python\Python312\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(data={'after':              # Timestamp       MMSI   Latitude  Longitude  ROT   SOG    COG  \
0    20…

[✓] CSV written → output\anomaly_edit.csv
